In [32]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

# Load in the Client Name data
# Make sure all names uppercase (there are some mixed instances)
pd.set_option('display.max_rows', 50)
vf = pd.read_csv('../input/cliente_tabla.csv',header=0)
vf['NombreCliente'] = vf['NombreCliente'].str.upper()
# Begin with a scan of the Client Name Data based on Top Frequency Client Names
# Notice there are a lot of Proper Names
vf['NombreCliente'].value_counts()[0:200]
def tfidf_score_list(vf2, list_len):
    from sklearn.feature_extraction.text import TfidfVectorizer
    v = TfidfVectorizer()

    vf2['New'] = 'na'
    a = " ".join(vf2['NombreCliente'])
    vf2['New'][0] = a

    tfidf = v.fit_transform(vf2['New'])

    feature_names = v.get_feature_names()

    freq = []
    doc = 0
    feature_index = tfidf[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
            freq.append((w.encode('utf-8'),s))
    
    del vf2['New']
    
    import numpy as np
    names = ['word','score']
    formats = ['S50','f8']
    dtype = dict(names = names, formats=formats)
    array = np.array(freq, dtype=dtype)

    b = np.sort(array, order='score')
    
    if list_len > len(b)+1:
        list_len = len(b)+1
    for i in range(1,list_len):
        print(b[-i])
tfidf_score_list(vf, 200)


# --- Begin Filtering for specific terms

# Note that the order of filtering is significant.
# For example: 
# The regex of .*ERIA.* will assign "FRUITERIA" to 'Eatery' rather than 'Fresh Market'.
# In other words, the first filters to occur have a bigger priority.

def filter_specific(vf2):
    
    # Known Large Company / Special Group Types
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*REMISION.*','Consignment')
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*WAL MART.*','.*SAMS CLUB.*'],'Walmart', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*OXXO.*','Oxxo Store')
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*CONASUPO.*','Govt Store')
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*BIMBO.*','Bimbo Store')

    

    # General term search for a random assortment of words I picked from looking at
    # their frequency of appearance in the data and common spanish words for these categories
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*COLEG.*','.*UNIV.*','.*ESCU.*','.*INSTI.*',\
                                                        '.*PREPAR.*'],'School', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*PUESTO.*','Post')
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*FARMA.*','.*HOSPITAL.*','.*CLINI.*'],'Hospital/Pharmacy', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*CAFE.*','.*CREMERIA.*','.*DULCERIA.*',\
                                                        '.*REST.*','.*BURGER.*','.*TACO.*', '.*TORTA.*',\
                                                        '.*TAQUER.*','.*HOT DOG.*',\
                                                        '.*COMEDOR.*', '.*ERIA.*','.*BURGU.*'],'Eatery', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].str.replace('.*SUPER.*','Supermarket')
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*COMERCIAL.*','.*BODEGA.*','.*DEPOSITO.*',\
                                                            '.*ABARROTES.*','.*MERCADO.*','.*CAMBIO.*',\
                                                        '.*MARKET.*','.*MART .*','.*MINI .*',\
                                                        '.*PLAZA.*','.*MISC.*','.*ELEVEN.*','.*EXP.*',\
                                                         '.*SNACK.*', '.*PAPELERIA.*', '.*CARNICERIA.*',\
                                                         '.*LOCAL.*','.*COMODIN.*','.*PROVIDENCIA.*'
                                                        ],'General Market/Mart'\
                                                       , regex=True)

    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*VERDU.*','.*FRUT.*'],'Fresh Market', regex=True)
    vf2['NombreCliente'] = vf2['NombreCliente'].replace(['.*HOTEL.*','.*MOTEL.*'],'Hotel', regex=True)

filter_specific(vf)

# --- Begin filtering for more general terms
# The idea here is to look for names with particles of speech that would
# not appear in a person's name.
# i.e. "Individuals" should not contain any participles or numbers in their names.
def filter_participle(vf2):
    vf2['NombreCliente'] = vf2['NombreCliente'].replace([
            '.*LA .*','.*EL .*','.*DE .*','.*LOS .*','.*DEL .*','.*Y .*', '.*SAN .*', '.*SANTA .*',\
            '.*AG .*','.*LAS .*','.*MI .*','.*MA .*', '.*II.*', '.*[0-9]+.*'\
    ],'Small Franchise', regex=True)
    
filter_participle(vf)


# Any remaining entries should be "Individual" Named Clients, there are some outliers.
# More specific filters could be used in order to reduce the percentage of outliers in this final set.
def filter_remaining(vf2):
    def function_word(data):
        # Avoid the single-words created so far by checking for upper-case
        if (data.isupper()) and (data != "NO IDENTIFICADO"): 
            return 'Individual'
        else:
            return data
    vf2['NombreCliente'] = vf2['NombreCliente'].map(function_word)
    
    
filter_remaining(vf)

/home/cankoklu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('no', 0.6888126004393861)
('identificado', 0.6849292193081505)
('la', 0.14990532034895288)
('el', 0.08328478631485127)
('abarrotes', 0.0800093233318993)
('de', 0.060769929775020375)
('maria', 0.046819424108208656)
('miscelanea', 0.038050812737366206)
('super', 0.03520332413080589)
('los', 0.02841652592336794)
('san', 0.025741686070920162)
('oxxo', 0.02257321668206611)
('del', 0.021668634477420134)
('garcia', 0.021376833766244014)
('hernandez', 0.02099506116912192)
('jose', 0.019835153342196838)
('lupita', 0.019261278610217134)
('gonzalez', 0.01775850494766011)
('martinez', 0.01764664800837593)
('lopez', 0.017150586799376526)
('mini', 0.015331695699712036)
('rodriguez', 0.014891562960354718)
('las', 0.01388971385198337)
('don', 0.012542567235386944)
('comodin', 0.01238450851683321)
('guadalupe', 0.012357760118308733)
('mary', 0.012270219904955897)
('jesus', 0.012024620973049328)
('ramirez', 0.011995440901931718)
('juan', 0.011966260830814104)
('casa', 0.011929785741917089)
('ag', 0.011

In [67]:
vf.head(15)

,Cliente_ID,NombreCliente
0,0,Individual
1,1,Oxxo Store
2,2,Individual
3,3,Small Franchise
4,4,Small Franchise
5,4,Small Franchise
6,5,Small Franchise
7,6,Individual
8,7,Small Franchise
9,8,Supermarket


In [70]:
vfddup= vf.drop_duplicates(subset=['Cliente_ID']).reset_index(inplace=True)

In [75]:
vf.head(15)

,Cliente_ID,NombreCliente
0,0,Individual
1,1,Oxxo Store
2,2,Individual
3,3,Small Franchise
4,4,Small Franchise
5,4,Small Franchise
6,5,Small Franchise
7,6,Individual
8,7,Small Franchise
9,8,Supermarket


In [79]:
ClientTypeList = vf['NombreCliente'].values
ClientTypeList[0:15]

array(['Individual', 'Oxxo Store', 'Individual', 'Small Franchise',
       'Small Franchise', 'Small Franchise', 'Small Franchise',
       'Individual', 'Small Franchise', 'Supermarket', 'Supermarket',
       'Small Franchise', 'Hospital/Pharmacy', 'Eatery', 'Individual'], dtype=object)

In [80]:
dumm = pd.get_dummies(ClientTypeList)

In [81]:
dumm.head(15)

,Bimbo Store,Consignment,Eatery,Fresh Market,General Market/Mart,Govt Store,Hospital/Pharmacy,Hotel,Individual,NO IDENTIFICADO,Oxxo Store,Post,School,Small Franchise,Supermarket,Walmart
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [85]:
#clientType_features = pd.concat([vf, dumm], axis=1)


clientType_features = pd.concat([vf.drop(['NombreCliente'], axis=1),dumm], axis=1)


In [93]:
clientType_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 930500 entries, 0 to 935361
Data columns (total 17 columns):
Cliente_ID             930500 non-null int64
Bimbo Store            930500 non-null int8
Consignment            930500 non-null int8
Eatery                 930500 non-null int8
Fresh Market           930500 non-null int8
General Market/Mart    930500 non-null int8
Govt Store             930500 non-null int8
Hospital/Pharmacy      930500 non-null int8
Hotel                  930500 non-null int8
Individual             930500 non-null int8
NO IDENTIFICADO        930500 non-null int8
Oxxo Store             930500 non-null int8
Post                   930500 non-null int8
School                 930500 non-null int8
Small Franchise        930500 non-null int8
Supermarket            930500 non-null int8
Walmart                930500 non-null int8
dtypes: int64(1), int8(16)
memory usage: 28.4 MB


In [90]:
clientType_features[['Bimbo Store','Consignment','Eatery','Fresh Market',
                     'General Market/Mart', 'Govt Store', 'Hospital/Pharmacy', 'Hotel', 'Individual',
                     'NO IDENTIFICADO', 'Oxxo Store', 'Post', 'School', 'Small Franchise', 'Supermarket',
                     'Walmart'
        ]]=clientType_features[['Bimbo Store','Consignment','Eatery','Fresh Market',
                     'General Market/Mart', 'Govt Store', 'Hospital/Pharmacy', 'Hotel', 'Individual',
                     'NO IDENTIFICADO', 'Oxxo Store', 'Post', 'School', 'Small Franchise', 'Supermarket',
                     'Walmart']].astype('int8')

In [92]:
clientType_features = clientType_features.drop_duplicates(subset=['Cliente_ID'])

In [95]:
clientType_features.to_csv('../input/clienttype_features.csv', index=False)